In [26]:
%reload_ext autoreload
%autoreload 2

# Use aligner_v7 kernel

import sys
add_paths = [
    "/fsx_0/user/tranx/rsync", # ALIGNER_PARENT_DIR
    "/fsx_0/user/tranx/rsync/llm_mm_aligner/replicated", # ALIGNER_PARENT_DIR/llm_mm_aligner/replicated
    "/fsx_0/shared/conda/aligner_20241030/python-packages"
]

for p in add_paths:
    if p not in sys.path:
        sys.path.append(p)
        
device = "cuda:0"

In [27]:
import torch
# from llm_mm_aligner.lib.encoders.metaclip_native_resolution_inference import (
#     MetaCLIP_NAVIT,
# )

from llm_mm_aligner.lib.encoders.metaclip_navit import MetaCLIP_NAVIT
from PIL import Image
from torch import nn
from torchvision import transforms

# Test load state_dict

In [13]:
# test loading
MODEL_H = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
MODEL_G = "/fsx_0/user/marcomonteiro/experiments/G/VEv0p2_G14_aws_final_256x8x98_next/checkpoints/epoch_450.pt"

sd_h = torch.load(MODEL_H, map_location="cpu")
sd_g = torch.load(MODEL_G, map_location="cpu")

In [20]:
_sd.keys()

dict_keys(['epoch', 'name', 'state_dict', 'optimizer'])

In [ ]:

_sd = sd_g
positional_embedding = None 

# if the checkpoint contains both visual and text modules, we only take the visual module
if 'state_dict' in _sd:
    print("yes")
    visual_module_prefix = "module.visual."
    visual_sd = {}
    for k, v in _sd["state_dict"].items():
        if k.startswith(visual_module_prefix):
            visual_sd[k.replace(visual_module_prefix, "")] = v
            
    if visual_sd:
        _sd = visual_sd
    else:
        _sd = _sd['state_dict']

# dropping positional_embedding and other unwanted layers
if ("positional_embedding" in _sd) and (positional_embedding is None):
    # Special case for VEv0.1 and VEv0.2 ckpt. The ckpt has positional embedding, but the model does not.
    _sd.pop("positional_embedding")
    print(
        "MetaCLIP_NAVIT dropping positional_embedding from checkpoint"
    )

drop_layers = [
    "mim_token",
    "proj",
    "attn_pool.probe",
    "attn_pool.attn.in_proj_weight",
    "attn_pool.attn.in_proj_bias",
    "attn_pool.attn.out_proj.weight",
    "attn_pool.attn.out_proj.bias",
    "attn_pool.layernorm.weight",
    "attn_pool.layernorm.bias",
    "attn_pool.mlp.c_fc.weight",
    "attn_pool.mlp.c_fc.bias",
    "attn_pool.mlp.c_proj.weight",
    "attn_pool.mlp.c_proj.bias",
]

loaded_layers = list(_sd.keys())
for k in loaded_layers:
    if k in drop_layers or k.endswith("attn.rope.freqs"):
        print(f"MetaCLIP_NAVIT dropping {k} from checkpoint")
        _sd.pop(k)

yes
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping mim_token from checkpoint
MetaCLIP_NAVIT dropping proj from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.0.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.1.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.2.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.3.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.4.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.5.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.6.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.7.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.8.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.9.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resb

# Init model

In [28]:
# MODEL_CHECKPOINT = "/fsx_0/checkpoints/clip/vev0/vit_h14_336_fair_vev0/fair_vev01.pt"
MODEL_CHECKPOINT = "/fsx_0/user/marcomonteiro/experiments/G/VEv0p2_G14_aws_final_256x8x98_next/checkpoints/epoch_450.pt"
TEST_IMAGE = "/fsx_0/dataset01/coco/val2014/COCO_val2014_000000485307.jpg"

# ViT-H
# metaclip_config = {
#     "ckpt_path": MODEL_CHECKPOINT,
#     "image_size": 336,  # image_size can also be 448, since the position embedding is rope_2d
#     "patch_size": 14,
#     "width": 1408,
#     "mlp_ratio": 4.0,
#     "layers": 34,
#     "heads": 16,
#     "load_ckpt": True,
#     "relative_pos_embed_type": "rope_2d",
# }

# ViT_G
metaclip_config = {
    "ckpt_path": MODEL_CHECKPOINT,
    "image_size": 392,  # image_size can also be 448, since the position embedding is rope_2d
    "patch_size": 14,
    "width": 1536,
    "mlp_ratio": 5.833333334,
    "layers": 50,
    "heads": 16,
    "load_ckpt": True,
    "relative_pos_embed_type": "rope_2d",
    "embed_cls_token": False
}

CLIP_MEAN = (0.4814546, 0.4578275, 0.40821073)
CLIP_STD = (0.2686295, 0.2613025, 0.2757711)

preprocess = transforms.Compose(
    [
        transforms.Resize(metaclip_config["image_size"]),
        transforms.CenterCrop(metaclip_config["image_size"]),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=(0.4814546, 0.4578275, 0.40821073),
            std=(0.2686295, 0.2613025, 0.2757711),
        ),
    ]
)

In [39]:
image = Image.open(TEST_IMAGE).convert("RGB")
preprocessed_image = preprocess(image).unsqueeze(0).to(device)

In [49]:
model = MetaCLIP_NAVIT(**metaclip_config).to(device)
model.eval()

MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping mim_token from checkpoint
MetaCLIP_NAVIT dropping proj from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.0.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.1.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.2.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.3.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.4.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.5.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.6.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.7.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.8.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblocks.9.attn.rope.freqs from checkpoint
MetaCLIP_NAVIT dropping transformer.resblock

MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping positional_embedding from checkpoint
MetaCLIP_NAVIT dropping 

MetaCLIP_NAVIT Visual Pretrained Model Missing keys: []
MetaCLIP_NAVIT Visual Pretrained Model Unexpected keys: []


MetaCLIP_NAVIT(
  (conv1): Conv2d(3, 1536, kernel_size=(14, 14), stride=(14, 14), bias=False)
  (patch_dropout): Identity()
  (ln_pre): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
  (transformer): Transformer(
    (resblocks): ModuleList(
      (0-49): 50 x ResidualAttentionBlock(
        (ln_1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (attn): AttentionXformer4x(
          (out_proj): Linear(in_features=1536, out_features=1536, bias=True)
        )
        (ls_1): Identity()
        (ln_2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
        (mlp): Sequential(
          (c_fc): Linear(in_features=1536, out_features=8960, bias=True)
          (gelu): QuickGELU()
          (c_proj): Linear(in_features=8960, out_features=1536, bias=True)
        )
        (ls_2): Identity()
      )
    )
  )
  (ln_post): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
)

In [50]:
with torch.no_grad():
    output = model(preprocessed_image)

output.shape

torch.Size([1, 784, 1536])

In [29]:
model

NameError: name 'model' is not defined